In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# dx = 1km; Np = 50M; Nz = 95
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#LOADING IN H5 VARIABLES
file_path = dir + 'Variable_Calculation/' + f'theta_e_{res}_{t_res}.h5'
th_e_ds1 = xr.open_dataset(file_path, phony_dims='sort',engine='h5netcdf')  # or engine='netcdf4'
th_e_ds1 = th_e_ds1.rename({
    'phony_dim_0': 'time',
    'phony_dim_1': 'zh',
    'phony_dim_2': 'yh',
    'phony_dim_3': 'xh'
})

In [ ]:
#CALCULATION FUNCTION
################################################################################
def MakeLapseRate(scalar):
    LapseRate = Ddz_3DStretch(scalar,data)
    return LapseRate

In [ ]:
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# dir2='/mnt/lustre/koa/scratch/air673/'
def initiate_array():
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=dir2 + 'Variable_Calculation/OUTPUT/' + 'theta_e_LapseRate'+f'_{res}_{t_res}'+'.h5'
    with h5py.File(out_file, 'a') as f:
        # Check if the dataset 'theta_e' already exists
        if 'theta_e' not in f:
            # Create a dataset with the full size for all time steps (initially empty)
            f.create_dataset('theta_e_LapseRate', 
                             (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                             maxshape=(None, z_size, y_size, x_size),  # Unlimited timesteps (can grow along time dimension)
                             dtype='float64', 
                             chunks=(1, z_size, y_size, x_size))  # Chunks for time axis to allow resizing

            
def add_timestep_at_index(timestep_data, index):
    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + 'theta_e_LapseRate'+f'_{res}_{t_res}'+'.h5'
    with h5py.File(out_file, 'a') as f:
        # Access the existing dataset 'theta_e'
        dataset = f['theta_e_LapseRate']
        
        # Assign the new timestep data at the specified index
        dataset[index] = timestep_data

In [ ]:
#RUNNING

In [ ]:
#MAKING ARRAY TO STORE THETA_E
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    theta_e_t = th_e_ds1['theta_e'].isel(time=t).data
    theta_e_LapseRate=MakeLapseRate(theta_e_t)
    add_timestep_at_index(theta_e_LapseRate, t)


#Fast for 1KM, 1.5 hours for 250M

In [ ]:
##########################################################################

In [ ]:
# #READING FINAL OUTPUT
# t=120
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# # dir2='/mnt/lustre/koa/scratch/air673/'
# with h5py.File(dir2 + 'Variable_Calculation/' + f'theta_e_LapseRate_{res}_{t_res}.h5', 'r') as f:
#     # Access the existing dataset 'MSE'
#     dataset = f['theta_e_LapseRate'][t]

In [ ]:
#TESTING
########################################################################

In [ ]:
# #READING FINAL OUTPUT
# t=120
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# # dir2='/mnt/lustre/koa/scratch/air673/'
# with h5py.File(dir2 + 'Variable_Calculation/' + f'theta_e_{res}_{t_res}.h5', 'r') as f:
#     # Access the existing dataset 'MSE'
#     dataset1 = f theta_e'][t]

In [ ]:
# #READING FINAL OUTPUT
# t=120
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# # dir2='/mnt/lustre/koa/scratch/air673/'
# with h5py.File(dir2 + 'Variable_Calculation/' + f'theta_e_LapseRate_{res}_{t_res}.h5', 'r') as f:
#     # Access the existing dataset 'MSE'
#     dataset2 = f['theta_e_LapseRate'][t]

In [ ]:
# z_level=5
# out=dataset2[z_level]
# plt.contourf(out)
# plt.colorbar()
# plt.title(f"z = {data['zh'][z_level].data*1000:.0f}m")

In [ ]:
# #TESTING CALCULATION INDEPENDENTLY
# zf=data['zf'].data
# subset1=dataset1[0:10];subset2=dataset2[0:10]
# # subset1=subset1[:,100,300]; subset2=subset2[:,100,300] ###Negative LapseRate
# subset1=subset1[:,199,510]; subset2=subset2[:,199,510] ###Positive LapseRate
# z_level=5
# #TRY ONE
# derivative = (subset1[z_level+1] - subset1[z_level-1]) / (2 * (zf[z_level] - zf[z_level-1]))
# print(derivative,subset2[z_level])

# #TRY TWO
# dz=np.diff(data['zf'].values)
# derivative = (subset1[z_level+1] - subset1[z_level-1]) / (2 * (dz[z_level]))
# print(derivative,subset2[z_level])